# Klampt Recipes

Note: some of these require Klampt version 0.10.0 or later.

In [ ]:
# Some values.  Change these around if you would like to test different files.

WORLD_FILE = '../data/athlete_plane.xml'
WORLD_FILE_MANIPULATOR = '../data/tx90cuptable.xml'
TRIMESH_FILE = '../data/objects/apc/kiva_pod/meshes/pod_lowres.stl'
POINT_CLOUD_FILE = '../data/objects/apc/genuine_joe_stir_sticks.pcd'
RGB_FILE = '../data/terrains/sunrgbdv2_data/sunrgbdv2_rgb.jpg'
DEPTH_FILE = '../data/terrains/sunrgbdv2_data/sunrgbdv2_depth.png'
EXTRINSICS_FILE = '../data/terrains/sunrgbdv2_data/sunrgbdv2_extrinsics.txt'

#this will pop up OpenGL windows like you'd see during desktop development, rather than try to embed in WebGL into this notebook
from klampt import vis
vis.init("PyQt")

## Visualization

### Debug a robot configuration

In [ ]:
from klampt import vis
from klampt import WorldModel
world = WorldModel()
world.loadFile(WORLD_FILE)
robot = world.robot(0)
qorig = robot.getConfig()
robot.randomizeConfig()
qtest = robot.getConfig()
qtest[0] += 1.0  #move to the right 1m
robot.setConfig(qorig)
#option 1: overlay configuration (in green) over world
vis.debug(qtest,{'color':(0,1,0,0.7)},world=world)


In [ ]:
#option 2: set the robot configuration
robot.setConfig(qtest)
vis.debug(world)

### Change the visualization window size and field of view

In [ ]:
from klampt import vis
from klampt import WorldModel
import math

w = WorldModel()
w.loadFile(WORLD_FILE)
vis.add("world",w)
vp = vis.getViewport()
#if you've previously saved a viewport to disk, you can load it here
#vp.load_file('saved_viewport.txt')
#Note: this API changed in klampt 0.10
vp.resize(1024,768)
vp.setFOV(math.radians(90))
vp.controller.tgt = [0,0,0.5]     #center point of the camera's rotation 
vp.controller.dist = 2.0   #bring the camera closer to the target
vp.controller.rot[0] = 0                #roll
vp.controller.rot[1] = -math.pi/4       #pitch
vp.controller.rot[2] = math.radians(10) #yaw
vis.setViewport(vp)
vis.loop()

#save current viewport to disk -- including all user changes
vp = vis.getViewport()
vp.save_file('saved_viewport.txt')

### Visually edit a robot trajectory

In [ ]:
from klampt import WorldModel
from klampt.io import resource
from klampt.model.trajectory import RobotTrajectory
world = WorldModel()
world.loadFile(WORLD_FILE)
robot = world.robot(0)
traj = RobotTrajectory(robot,[0],[robot.getConfig()])
#if you're loading a trajectory from disk, uncomment this
#traj = resource.get('some_trajectory.traj') 
(ok_clicked, traj) = resource.edit('some_trajectory',traj,world=world)
#if you're saving the trajectory to disk, uncomment this
#if ok_clicked:
#   resource.set('some_trajectory.traj',traj)

### Overlay Klampt items on a camera image

Example gives intrinsics as JSON, extrinsics as a numpy array, and image as a PNG file loaded with OpenCV.  (Not runnable here)

In [ ]:
from klampt import vis, WorldModel
from klampt.math import se3
#load intrinsics as a JSON file containing 'fx','fy','cx','cy'
import json 
with open('intrinsics.json','r') as f:
    intrinsics = json.load(f)
#load extrinsics as a numpy array
import numpy as np
extrinsics = np.loadtxt('extrinsics.npy')
#load the camera image
import cv2
image = cv2.imread('image.png')
h = image.shape[0]
w = image.shape[1]

#set up the Klampt viewport
vis.init()
vp = vis.getViewport()
vp.w = w
vp.h = h
vp.fx = intrinsics['fx']
vp.fy = intrinsics['fy']
vp.cx = intrinsics['cx']
vp.cy = intrinsics['cy']
vp.set_transform(se3.from_ndarray(extrinsics))
vis.setViewport()
#draw the image
image_rgb = image[:,:,::-1]  #convert from BGR to RGB
vis.scene().setBackgroundImage(image_rgb)
world = WorldModel()
world.loadFile(WORLD_FILE)
vis.add(world=world)   #add other things to the scene
vis.loop()

### Draw a point cloud as a collection of spheres

In [ ]:
from klampt import PointCloud,Geometry3D
from klampt.model.create import primitives
from klampt.model.geometry import point_cloud_colors
from klampt import vis

vis.clear()
g = Geometry3D()
if not g.loadFile(POINT_CLOUD_FILE):
    raise IOError("Unable to load point cloud")
pc = g.getPointCloud()
#draw the point cloud as a collection of spheres
pts = pc.points
colors = point_cloud_colors(pc,format=('r','g','b'))
r = 0.001
for i in range(len(pts)):
    if i % 100 != 0:  #if you have lots of points, you should only draw a subsample
        continue
    color = colors[i].tolist() if colors is not None else (1,0,0,0.5)
    vis.add('point'+str(i),primitives.sphere(r,pts[i],type='GeometricPrimitive'),color=color,hide_label=True)
vis.autoFitCamera()
vis.loop()

## Geometry

### Add a sphere terrain to a world

In [ ]:
from klampt import WorldModel
from klampt.model.create import primitives
world = WorldModel()
terrain = world.makeTerrain('ball')
#you can also use type='TriangleMesh' to represent the sphere as a triangle mesh 
terrain.geometry().set(primitives.sphere(radius=0.5,center=(1.5,0,0.5),type='GeometricPrimitive'))

In [ ]:
# visualize the above
from klampt import vis
terrain.appearance().setColor(0.75,0.25,0.25,1)
vis.add("world",world)
vis.loop()

### Add a sphere rigid object to a world

In [ ]:
from klampt import WorldModel
from klampt.model.create import primitives
from klampt.math import so3
world = WorldModel()
obj = world.makeRigidObject('ball')
#you can also use type='TriangleMesh' to represent the sphere as a triangle mesh 
obj.geometry().set(primitives.sphere(radius=0.5,type='GeometricPrimitive'))
obj.setTransform(so3.identity,(1.5,0,0.5))

In [ ]:
# visualize the above
from klampt import vis
terrain.appearance().setColor(0.75,0.25,0.25,1)
vis.add("world",world)
vis.loop()


### Sample uniformly spaced points on the surface of a mesh or other geometry


In [ ]:
from klampt import Geometry3D
mesh = Geometry3D()
mesh.loadFile(TRIMESH_FILE)
resolution = 0.01  #how far you want the points to be spaced.  0 samples vertices only
pc = mesh.convert('PointCloud',resolution)
points = pc.getPointCloud().points   #the output is a numpy array of shape (N,3)
#these are in the local frame of the geometry. If you want the world frame, use
#from klampt.math import so3
#points = np.dot(points,so3.ndarray(mesh.getTransform()[0]).T) + mesh.getTransform()[1]

In [ ]:
#visualize the point cloud
from klampt.vis.colorize import colorize
colorize(pc,'z')
vis.debug(pc)

### Sample at random from the surface of a mesh or other geometry


In [ ]:
from klampt import Geometry3D
from klampt.model.geometry import sample_surface
mesh = Geometry3D()
mesh.loadFile(TRIMESH_FILE)
N = 1000   #the number of points to sample
points = sample_surface(mesh,N,want_normals=False)   #the output is a numpy array of shape (N,3)
#these are in the local frame of the geometry. If you want the world frame, use
#from klampt.math import so3
#points = np.dot(points,so3.ndarray(mesh.getTransform()[0]).T) + mesh.getTransform()[1]

In [ ]:
#visually debug the sampled points
from klampt.vis.colorize import colorize
from klampt.io.numpy_convert import from_numpy
import numpy as np

pc = from_numpy(np.array(points),'PointCloud')
colorize(pc,'z')
vis.debug(pc)

### Create a depth map from an RGB and depth image
Requires OpenCV to load images from file.

In [ ]:
import cv2
import numpy as np
from klampt.math import se3,so3
from klampt import Heightmap,Geometry3D
from klampt import vis

hm = Heightmap()
xfov = 1.0166577814521098
yfov = 0.7916406885147462
height_range = [0.0,4.0]

img = cv2.imread(DEPTH_FILE,cv2.IMREAD_UNCHANGED)  #16-bit depth file
print("DEPTH IMAGE SIZE",img.shape,"DTYPE",img.dtype,"DEPTH RANGE",np.min(img),np.max(img))
hm.setHeightImage(img, height_range[1]/(65535.0))
assert not hm.isPerspective()
img = cv2.imread(RGB_FILE)
print("RGB IMAGE SIZE",img.shape,"DTYPE",img.dtype)
hm.setColorImage(img)

hm.setFOV(xfov,yfov)
assert hm.isPerspective()

extrinsics = np.loadtxt(EXTRINSICS_FILE)
pose = so3.from_ndarray(extrinsics[:3,:3]),extrinsics[:3,3]
#add an extra y-z flip
pose = se3.mul((so3.rotation([1,0,0],-np.pi/2),[0]*3),pose)

geom = Geometry3D()
geom.setHeightmap(hm)
geom.setCurrentTransform(*pose)
print(pose)
print("Viewport dimensions",hm.viewport.x,hm.viewport.y,hm.viewport.w,hm.viewport.h)
print("Viewport intrinsics",hm.viewport.fx,hm.viewport.fy,hm.viewport.cx,hm.viewport.cy)
print("BB",geom.getBB())
vis.add('origin',se3.identity())
vis.add('pose',pose)
vis.add('hm',geom)
#vis.show()


In [ ]:
#To view as a PointCloud and save as a PCD
pc = geom.convert('PointCloud')
vis.debug(pc)
if pc.saveFile('test.pcd'):
    print("Saved file to test.pcd")
else:
    print("Error saving to test.pcd")

## Robot Models

### Showing a robot's reachable workspace

See ``Klampt-examples/Python3/demos/model/workspace_test.py``

### Converting all geometries to implicit surfaces

In [ ]:
from klampt import WorldModel

world = WorldModel()
world.loadFile(WORLD_FILE)
robot = world.robot(0)
for l in robot.links:
    g = l.geometry()
    if g.empty():
        continue
    g.set(g.convert('ImplicitSurface',0.01))  #second argument is a resolution parameter. Try 0.005 for better conversion quality
vis.debug(world)

### Creating a floating rigid body robot from a geometry

In [ ]:
from klampt import WorldModel
from klampt.model.create import moving_base_robot
from klampt import vis

world = WorldModel()
moving_base_robot.make(TRIMESH_FILE,world)
vis.add('world',world)
vis.edit(('world',world.robot(0).name))
vis.loop()

## Inverse Kinematics

### Solve for collision-free IK


In [ ]:
from klampt import WorldModel
from klampt.model.collide import WorldCollider
from klampt.model import ik
from klampt.math import vectorops,so3

world = WorldModel()
world.loadFile(WORLD_FILE_MANIPULATOR)  #point this to the robot file
robot = world.robot(0)

#set up the desired end effector transform
Rlink = so3.from_rpy([0,0,0])    #roll pitch yaw
pworld = [1,0,0.3]
#local point on the robot that you'd like to match to t
plocal = [0,0,0]
#set up IKObjective -- arguments (R,t) are the desired transform of the
#link, and we want R*plocal+t=pworld.  So, we set t=pworld - R*plocal
goal = ik.objective(robot.link(robot.numLinks()-1),R=Rlink,t=vectorops.sub(pworld,so3.apply(Rlink,plocal))) 

#set up feasibility tests.  Joint limits are automatically respected
collider = WorldCollider(world)
#collider.ignoreCollision(...)  #can be used to ignore collision checking between objects
def checkCollisions():
    #robot model will be set to the currently considered configuration
    for i in collider.robotTerrainCollisions(robot):
        return False
    for i in collider.robotObjectCollisions(robot):
        return False
    for i in collider.robotSelfCollisions(robot):
        return False
    return True

In [ ]:
# If you need to check whether your IK goal is reasonable, you can debug it
vis.debug(goal,world=world)

In [ ]:

#you may set up an initial guess here.  solve_global will use this as the guess for the first solve
robot.setConfig([0]*robot.numLinks())
#solve the IK problem
res = ik.solve_global(goal,iters=100,tol=1e-3,feasibilityCheck=checkCollisions)
if res:
    print("Collision-free IK succeeded!")
else:
    print("Collision-free IK failed")
vis.debug(goal,world=world)

## Sensors

### Create a camera and take snapshots

In [ ]:
from klampt import WorldModel
from klampt.model import sensing
from klampt.math import se3,so3
import math
world = WorldModel()
world.loadFile(WORLD_FILE) #load a world / robot file
robot = world.robot(0)
cam = robot.addSensor("my_world_camera","CameraSensor")
tcamera = [0,-2,1.0]
Rcamera = so3.from_rpy([math.pi/2,math.pi,math.pi]) #z points forward, x points right, y points down
xform = Rcamera,tcamera   
cam_link = None   #can also try, e.g., robot.link(robot.numLinks()-1)
sensing.set_sensor_xform(cam,xform,cam_link)
cam.setSetting('xres',str(640))
cam.setSetting('yres',str(480))
cam.setSetting('xfov',str(math.radians(75.0)))
cam.setSetting('yfov',str(math.radians(60.0)))
cam.setSetting('zmin',str(0.1))
cam.setSetting('zmax',str(10.0))
cam.setSetting('rgb',str(1))
cam.setSetting('depth',str(1))  #turn this to 0 to turn off depth sensing

#Note that camera simulation will be ugly until an OpenGL window has been created.
# cam.kinematicSimulate(world,1.0/30.0)  #< this won't work well.
# rgb, depth = sensing.camera_to_images(cam)

#Workaround to use Klampt's vis to take pictures using OpenGL rendering
from klampt import vis
import time
rgb, depth = None, None
def take_snapshot():
    global rgb,depth
    if vis.shown():   #opengl will have been initialized
        vis.lock()
        cam.kinematicSimulate(world,1.0/30.0)
        vis.unlock()
        rgb, depth = sensing.camera_to_images(cam)
        vis.show(False)  #kill once the image is taken
    else:
        print("Waiting for OpenGL to initialize")
        time.sleep(0.01)
        vis.threadCall(take_snapshot)

vis.add("world",world)
vis.add("Tcamera",xform)
vis.show()
vis.threadCall(take_snapshot)
while vis.shown():
    time.sleep(0.01)

#save images
if rgb is not None:
    from PIL import Image
    import numpy as np
    Image.fromarray(rgb).save('rgb.png')
    print("Depth range",np.min(depth),np.max(depth))
    dmax = 4.0
    depth8 = np.minimum(depth*256/dmax,255).astype(np.uint8)
    print("Depth image range",np.min(depth8),np.max(depth8))
    Image.fromarray(depth8).save('depth.png')

